## **Import Libraries**

In [61]:
import torch
import csv
import torch.nn as nn
import random
import numpy as np
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable 
from tqdm import tqdm
import random
import torch.nn.functional as F

## **SET DEVICE (CPU / GPU)**

In [62]:
# Set Device which to use
def set_device():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
    return device

device = set_device()
print(device)

cuda


## **LOAD DATA**

In [63]:
def load_data(lan = 'hin'):
    train_data_path = '/kaggle/input/vocabs/Dataset/hin/hin_train.csv'
    test_data_path = '/kaggle/input/vocabs/Dataset/hin/hin_test.csv'
    val_data_path = '/kaggle/input/vocabs/Dataset/hin/hin_valid.csv'
    
    train_x = []
    train_y = []
    with open(train_data_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # next(csv_reader)  # Skip the header row if it exists
        for row in csv_reader:
            train_x.append(row[0] + '$')
            train_y.append('#' + row[1] + '$')
       
    
    val_x, val_y = [], []
    with open(val_data_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # next(csv_reader)  # Skip the header row if it exists
        for row in csv_reader:
            val_x.append(row[0] + '$')
            val_y.append('#' + row[1] + '$')
    
    test_x, test_y = [], []
    with open(test_data_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        # next(csv_reader)  # Skip the header row if it exists
        for row in csv_reader:
            test_x.append(row[0] + '$')
            test_y.append('#' + row[1] + '$')

    # Conversion in NP Array
    train_x, train_y = np.array(train_x), np.array(train_y)
    val_x, val_y = np.array(val_x), np.array(val_y)
    test_x, test_y = np.array(test_x), np.array(test_y)

    #get size of data
    max_decoder_length_train = np.max(np.array([len(s) for s in train_y]))
    max_decoder_length_val = np.max(np.array([len(s) for s in val_y]))
    max_decoder_length_test = np.max(np.array([len(s) for s in test_y]))
    max_decoder_length = max(max_decoder_length_train, max(max_decoder_length_val, max_decoder_length_test))

    max_encoder_length_train = np.max(np.array([len(s) for s in train_x]))
    max_encoder_length_val = np.max(np.array([len(s) for s in val_x]))
    max_encoder_length_test = np.max(np.array([len(s) for s in test_x]))
    max_encoder_length = max(max_encoder_length_train, max(max_encoder_length_val, max_encoder_length_test))


    print(max_decoder_length)
    print(max_encoder_length)

    

# Find the maximum length
    return_res = {
        "train_x" : train_x,
        "train_y" : train_y,
        "val_x" : val_x,
        "val_y" : val_y,
        "test_x" : test_x,
        "test_y" : test_y,
        "max_decoder_length" : max_decoder_length,
        "max_encoder_length" : max_encoder_length,
    }

    return return_res


res = load_data("hin")
tx = res["train_x"]
ty = res["train_y"]
vx = res["val_x"]
vy = res["val_y"]
tex = res["test_x"]
tey = res["test_y"]
max_decoder_length = res["max_decoder_length"]
max_encoder_length = res["max_encoder_length"]
train_len = tx.shape[0]
val_len = vx.shape[0]


22
27


In [64]:
def create_corpus(train_x, train_y, val_x, val_y, test_x, test_y):

    english_vocab = "#$abcdefghijklmnopqrstuvwxyz"
    input_corpus_dict = {}
    input_corpus_dict[''] = 0
    index = 1
    for characters in english_vocab:
        input_corpus_dict[characters] = index
        index += 1
    
    output_vocab = set()
    for word in train_y:
        for characters in word:
            output_vocab.add(characters)
    for word in val_y:
        for characters in word:
            output_vocab.add(characters)
    for word in test_y:
        for characters in word:
            output_vocab.add(characters)
    output_vocab.add('')
    sorted_output_vocab = sorted(output_vocab)

    output_corpus_dict = {}
    index = 0
    for characters in sorted_output_vocab:
        output_corpus_dict[characters] = index
        index += 1

    reversed_input_corpus = {value: key for key, value in input_corpus_dict.items()}
    reversed_output_corpus = {value: key for key, value in output_corpus_dict.items()}
    return_dict = {
        "input_corpus_length" : len(english_vocab),
        "output_corpus_length" : len(output_vocab),
        "input_corpus_dict" : input_corpus_dict,
        "output_corpus_dict" : output_corpus_dict,
        "reversed_input_corpus" : reversed_input_corpus,
        "reversed_output_corpus" : reversed_output_corpus
    }
    
    return return_dict

res_dict = create_corpus(tx, ty, vx, vy, tex, tey)
print(res_dict["input_corpus_length"])
print(res_dict["output_corpus_length"])
print(res_dict["input_corpus_dict"])
print(res_dict["output_corpus_dict"])
print(res_dict["reversed_input_corpus"])
print(res_dict["reversed_output_corpus"])

input_corpus_length = res_dict["input_corpus_length"]
output_corpus_length = res_dict["output_corpus_length"]
output_corpus_dict = res_dict["output_corpus_dict"]
input_corpus_dict = res_dict["input_corpus_dict"]
reversed_output_corpus = res_dict["reversed_output_corpus"]
reversed_input_corpus = res_dict["reversed_input_corpus"]
        

28
68
{'': 0, '#': 1, '$': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{'': 0, '#': 1, '$': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'ळ': 46, 'व': 47, 'श': 48, 'ष': 49, 'स': 50, 'ह': 51, '़': 52, 'ऽ': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'ॅ': 60, 'े': 61, 'ै': 62, 'ॉ': 63, 'ॊ': 64, 'ो': 65, 'ौ': 66, '्': 67}
{0: '', 1: '#', 2: '$', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n'

In [65]:
def create_tensor(tx, ty, vx, vy, res, dict):
    max_len = max(res["max_encoder_length"], res["max_decoder_length"])
    train_input = np.zeros((max_len, train_len), dtype = 'int64')
    train_output = np.zeros((max_len, train_len), dtype = 'int64')
    val_input = np.zeros((max_len, val_len), dtype = 'int64')
    val_output = np.zeros((max_len, val_len), dtype = 'int64')

    print(dict["input_corpus_dict"])
    print(dict["output_corpus_dict"])
    word_count = 0
    for words in tx:
        index = 0
        for chars in words:
            # print(chars)
            train_input[index, word_count] = dict["input_corpus_dict"][chars]
            index += 1
        word_count += 1

    word_count = 0
    for words in ty:
        index = 0
        for chars in words:
            # print(chars)
            train_output[index, word_count] = dict["output_corpus_dict"][chars]
            index += 1
        word_count += 1

    word_count = 0
    for words in vx:
        index = 0
        for chars in words:
            # print(chars)
            val_input[index, word_count] = dict["input_corpus_dict"][chars]
            index += 1
        word_count += 1

    word_count = 0
    for words in vy:
        index = 0
        for chars in words:
            # print(chars)
            val_output[index, word_count] = dict["output_corpus_dict"][chars]
            index += 1
        word_count += 1


    # Convert in tensor
    train_input = torch.tensor(train_input)
    train_output = torch.tensor(train_output)
    val_input = torch.tensor(val_input)
    val_output = torch.tensor(val_output)

    return_dict = {
        "train_input" : train_input,
        "train_output" : train_output,
        "val_input" : val_input,
        "val_output" : val_output,
    }
    return return_dict

In [66]:
tensors = create_tensor(tx, ty, vx, vy, res, res_dict)
train_input = tensors["train_input"]
val_input = tensors["val_input"]
train_output = tensors["train_output"]
val_output = tensors["val_output"]
print(train_input.shape)
print(train_output.shape)

{'': 0, '#': 1, '$': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{'': 0, '#': 1, '$': 2, 'ँ': 3, 'ं': 4, 'ः': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'ळ': 46, 'व': 47, 'श': 48, 'ष': 49, 'स': 50, 'ह': 51, '़': 52, 'ऽ': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'ॅ': 60, 'े': 61, 'ै': 62, 'ॉ': 63, 'ॊ': 64, 'ो': 65, 'ौ': 66, '्': 67}
torch.Size([27, 51200])
torch.Size([27, 51200])


## **SEQ2SEQ**

In [67]:
class Encoder(nn.Module):
    def __init__(self, PARAM):
        super(Encoder, self).__init__()
        self.input_size = PARAM["input_size"]
        self.embedding_size = PARAM["embedding_size"]
        self.hidden_size = PARAM["hidden_size"]
        self.num_layers = PARAM["num_layers"]
        self.drop_prob = PARAM["drop_prob"]
        self.cell_name = PARAM["cell_name"]

        self.dropout = nn.Dropout(self.drop_prob)
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        
        if self.cell_name == "RNN":
            self.cell = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob)
        elif self.cell_name == "LSTM":
            self.cell = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob)
        elif self.cell_name == "GRU":
            self.cell = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob)
            
    def forward(self, x):
        embedding = self.embedding(x)
#         embedding = F.relu(embedding)
        drops = self.dropout(embedding)
        outputs, (hidden, cells)= self.cell(drops)
        return hidden, cells
    
class Decoder(nn.Module):
    def __init__(self, PARAM):
        super(Decoder, self).__init__()
        self.input_size = PARAM["input_size"]
        self.embedding_size = PARAM["embedding_size"]
        self.hidden_size = PARAM["hidden_size"]
        self.output_size = PARAM["output_size"]
        self.num_layers = PARAM["num_layers"]
        self.drop_prob = PARAM["drop_prob"]
        self.cell_name = PARAM["cell_name"]
        self.dropout = nn.Dropout(self.drop_prob)
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)

        if self.cell_name == "RNN":
            self.cell = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob)
        elif self.cell_name == "LSTM":
            self.cell = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob)
        elif self.cell_name == "GRU":
            self.cell = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.drop_prob)
        
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x, hidden, cells):
        x = x.unsqueeze(0)
        embedding = self.embedding(x)
#         embedding = F.relu(embedding)
        drops = self.dropout(embedding)
        outputs, (hidden, cells) = self.cell(drops, (hidden, cells))
        predictions = self.fc(outputs).squeeze(0)
        predictions = F.log_softmax(predictions, dim = 1)
        return predictions, hidden, cells

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, target, teacher_forcing_ratio = 0.5):
        batch_size = src.shape[1]
        target_len = target.shape[0]

        target_vocab_size = len(output_corpus_dict)
        
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        
        x = target[0]
        encoder_hidden, cell = self.encoder(src) 
    
        for t in range(1, target_len):
            output, encoder_hidden, cell = self.decoder(x, encoder_hidden, cell)
            outputs[t] = output
            best_guess = output.argmax(1)

            x = best_guess if random.random() >= teacher_forcing_ratio else target[t]
        
        return outputs
        


## **Training**

In [68]:
def evaluate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    correct_pred = 0
    total_pred = 0

    with torch.no_grad():
        for src, target in val_loader:
            src = src.to(device)
            target = target.to(device)

            output = model(src, target, 0)
            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            target = target[1:].view(-1)

            loss = criterion(output, target)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1)
            total_loss += loss.item()

            predicted = output.argmax(dim=1)
            correct_pred += (predicted == target).sum().item()
            total_pred += target.numel()
        
    val_loss = total_loss / len(val_loader)
    val_accuracy = correct_pred / total_pred

    return val_loss, val_accuracy
    
def train(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_pred = 0
        total_pred = 0

        for src, target in tqdm(train_loader, desc = f"Epoch {epoch + 1}/{epochs}"):
            src = src.device()
            target = target.device()

            optimizer.zero_grad()
            output = model(src, target)
            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            target = target[1:].view(-1)

            loss = criterion(output, target)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Calculate accuracy
            predicted = output.argmax(dim=1)
            correct_pred += (predicted == target).sum().item()
            total_pred += target.numel()
        train_loss = total_loss / len(train_loader)
        train_accuracy = correct_pred / total_pred
        val_loss, val_accuracy = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f} Val Accuracy: {val_accuracy:.4f}")    

    return model



In [69]:
PARAM_Encoder = {
    "input_size" : len(input_corpus_dict),
    "embedding_size" : 256,
    "hidden_size" : 512,
    "num_layers" : 2,
    "drop_prob" : 0.5,
    "cell_name" : "LSTM"
}

PARAM_Decoder = {
    "input_size" : len(output_corpus_dict),
    "embedding_size" : 256,
    "hidden_size" : 512,
    "output_size" : len(output_corpus_dict),
    "num_layers" : 2,
    "drop_prob" : 0.5,
    "cell_name" : "LSTM"
}


eta = 0.001
epochs = 10


encoder = Encoder(PARAM_Encoder).to(device)
decoder = Decoder(PARAM_Decoder).to(device)

model = Seq2Seq(encoder, decoder).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=eta)



# print(train_input.shape, train_output.shape)
# train_data = TensorDataset(train_input, train_output)
# train_loader = DataLoader(train_data, shuffle=True)

train_ds_y, train_ds_x = torch.split(train_output, 32, dim = 1), torch.split(train_input, 32, dim = 1)
val_ds_y, val_ds_x = torch.split(val_output, 32, dim = 1), torch.split(val_input, 32, dim = 1)

correct_pred = 0

for epoch in range(epochs):
  print(f"Epoch :: {epoch}/{epochs}")
  total_words = 0
  correct_pred = 0
  total_loss = 0
  model.train()
  with tqdm(total=len(train_ds_x), desc='Training') as pbar:
    for i, (x, y) in enumerate(zip(train_ds_x, train_ds_y)):
      target, inp_data = y.to(device), x.to(device)
      output = model(inp_data, target)
      pad_mask = (target != output_corpus_dict[''])  # Replace PAD_INDEX with your actual padding character index
      non_pad_targets = target[pad_mask]  # Select non-padding elements
      non_pad_outputs = output[pad_mask].reshape(-1, output.shape[2])  # Select corresponding outputs
      loss = criterion(non_pad_outputs, non_pad_targets)
      optimizer.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
      optimizer.step()
      total_words += non_pad_targets.size(0)
      correct_pred += torch.sum(torch.argmax(non_pad_outputs, dim=1) == non_pad_targets).item()
      total_loss += loss.item()
      pbar.update(1)

  accuracy = correct_pred / total_words
  avg_loss = total_loss / len(train_ds_x)  # Average loss per batch

  model.eval()
  with torch.no_grad():
    val_total_loss = 0
    val_total_words = 0
    val_correct_pred = 0
    with tqdm(total=len(val_ds_x), desc = 'Validation') as pbar:
      for x_val, y_val in zip(val_ds_x, val_ds_y):
        target_val, inp_data_val = y_val.to(device), x_val.to(device)
        output_val = model(inp_data_val, target_val)
        pad_mask = (target_val != output_corpus_dict[''])
        non_pad_targets = target_val[pad_mask]
        non_pad_outputs = output_val[pad_mask].reshape(-1, output_val.shape[2])
        val_loss = criterion(non_pad_outputs, non_pad_targets)
        val_total_loss += val_loss.item()
        val_total_words += non_pad_targets.size(0)
        val_correct_pred += torch.sum(torch.argmax(non_pad_outputs, dim=1) == non_pad_targets).item()
        pbar.update(1)

    val_accuracy = val_correct_pred / val_total_words
    val_avg_loss = val_total_loss / len(val_ds_x)
    print(f"Epoch : {epoch+1} Train Accuracy: {accuracy*100:.4f}, Train Loss: {avg_loss:.4f} Validation Accuracy: {val_accuracy*100:.4f}, Validation  Loss: {val_avg_loss:.4f}")


# for epoch in range(epochs):
#     print(f"Epoch :: {epoch}/{epochs}")
#     model.train()
#     total_loss = 0
#     correct_pred = 0
#     total_pred = 0

#     for i, (x, y) in enumerate(zip(train_ds_x, train_ds_y)):
#         # Get input and targets and move to device
#         target, inp_data = y.to(device), x.to(device)
        
#         # Forward propagation
#         output = model(inp_data, target)
#         target, output = target[1:].reshape(-1),  output[1:].reshape(-1, output.shape[2])
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss = criterion(output, target)
#         loss.backward()
        
#         # Clip gradients to avoid exploding gradients
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        
#         # Update parameters
#         optimizer.step()
        
#         # Update total words processed
#         total_pred += target.size(0)
        
#         # Calculate number of correct predictions
#         correct_pred += torch.sum(torch.argmax(output, dim=1) == target).item()

#         total_loss += loss.item()
#     accuracy = correct_pred / total_pred
#     avg_loss = total_loss / len(train_ds_x)  # Average loss per batch
#     print(f"Accuracy: {accuracy:.4f}, Average Loss: {avg_loss:.4f}")    



Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(29, 256)
    (cell): LSTM(256, 512, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(68, 256)
    (cell): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=512, out_features=68, bias=True)
  )
)
Epoch :: 0/10


Validation: 100%|██████████| 128/128 [00:01<00:00, 76.85it/s]


Epoch : 1 Train Accuracy: 33.5873, Train Loss: 2.4873 Validation Accuracy: 57.2322, Validation  Loss: 1.5213
Epoch :: 1/10


Validation: 100%|██████████| 128/128 [00:01<00:00, 77.71it/s]


Epoch : 2 Train Accuracy: 57.9354, Train Loss: 1.4571 Validation Accuracy: 65.6641, Validation  Loss: 1.2509
Epoch :: 2/10


...

Epoch : 8 Train Accuracy: 73.0051, Train Loss: 0.9690 Validation Accuracy: 72.0508, Validation  Loss: 1.0544
Epoch :: 8/10


Validation: 100%|██████████| 128/128 [00:01<00:00, 70.04it/s]


Epoch : 9 Train Accuracy: 73.8266, Train Loss: 0.9414 Validation Accuracy: 71.6481, Validation  Loss: 1.0696
Epoch :: 9/10


Validation: 100%|██████████| 128/128 [00:01<00:00, 73.41it/s]

Epoch : 10 Train Accuracy: 74.4492, Train Loss: 0.9216 Validation Accuracy: 72.1451, Validation  Loss: 1.0524


In [110]:
def predict(model, PARAM):
    word = PARAM["word"]
    input_char_index = PARAM["input_corpus_dict"]
    output_char_index = PARAM["output_corpus_dict"]
    reverse_target_char_index = PARAM["reversed_output_corpus"]
    
    data, word_t = np.zeros((len(input_char_index),1), dtype= int), ''
    t_z = 0
    for t, char in enumerate(word):
        data[t, 0] = input_char_index[char]
    t_z = t+1   
    data[t_z :,0] = input_char_index["$"]
    
    data = torch.tensor(data,dtype = torch.int64).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(data)
    out_t = output_char_index['#']    
    out_chr_reshape = np.array(out_t).reshape(1,)    
    x = torch.tensor(out_chr_reshape).to(device)

    for t in range(1, len(output_char_index)):
        
        output, hidden, cell = model.decoder(x, hidden, cell)
        # best_guess = output.argmax(1)
        ch = reverse_target_char_index[output.argmax(1).item()]
        
        if ch != '$':
            word_t = word_t+ch
        else:
            break

    return word_t


# Define the input sequence
PARAM_PRED = {
    "word" : "chalo",
    "input_corpus_dict" : input_corpus_dict,
    "output_corpus_dict" : output_corpus_dict,
    "reversed_output_corpus" : reversed_output_corpus,
}


# Predict the output sequence
for word, target in zip(vx, vy):
    PARAM_PRED["word"] = word
    output_sequence = predict(model, PARAM_PRED)
    print(word, target, output_sequence)

output_sequence = predict(model, PARAM_PRED)
print("Input Sequence:", PARAM_PRED["word"])
print("Predicted Output Sequence:", output_sequence)

jaisawal$ #जयसवाल$ जैससववलललल
bajai$ #बजाई$ बजजयय
sanghthan$ #संघठन$ संघघथनननन
haiwaan$ #हैवान$ हइववयंं
nilgiri$ #नीलगिरि$ निलगिगररर
drutgrami$ #द्रुतग्रामी$ द्र्रटटररररआआआआआआआआ
jhadapon$ #झड़पों$ झडडाोंंंं
nakronda$ #नकरोंदा$ नककररोडडड
eesl$ #ईईएसएल$ ईईएएएएए
bachta$ #बचता$ बचचटटटटट
vodka$ #वोडका$ वोडकककक
kristian$ #क्रिस्टन$ क्रिस्शशयन
guanajuata$ #गुआनाजुआटा$ गुणााजजततत
shershi$ #शेरशी$ शेरशष
mannaki$ #मन्नाकी$ मनननकककीक
jimmaa$ #जिम्मा$ जिममममा
yun$ #यूं$ युन
baig$ #बेग$ बैईग
marsi$ #मारसी$ मारसस
pus$ #पूस$ पसयस
charmful$ #चार्मफुल$ चारममफफललल
sriperu$ #श्रीपेरू$ श्रीरेरर
russio$ #रशियो$ रसससय
ihfl$ #आईएचएफएल$ आईएएएएएएएए
ganinath$ #गणिनाथ$ गणनननथथथथथ
bandikui$ #बंदीकुई$ बंंदीककययआआआआआआआ
lasalle$ #लासाले$ लालाललल
namdan$ #नामदान$ नममददनन
decaine$ #डिकेन$ डिकाकनन
redesign$ #रीडिजाइन$ रेडेसिसंज
holobon$ #होलोबोन$ होलोबोबो
ilakon$ #इलाकों$ इललककों
aashcharyajanak$ #आश्चर्यजनक$ आशशचचरररययययनककक
krus$ #क्रूस$ क्रसययययय
option$ #आप्शन$ ऑप्टशशनननन
mahal$ #महल$ महहलल
recapitalization$ #रिकैप

## **BEAM SEARCH**

In [ ]:
def beam_search(PARAM, model, word, device):
    dtype = torch.int32
    data, word_t = np.zeros((PARAM["max_encoder_length"], 1), dtype='int32'), ""
    
    index = 0
    for character in word:
        data[index, 0] = PARAM["input_corpus_dict"][character]
        index += 1
    
    data = torch.tensor(data, dtype=torch.int32).to(device)

    with torch.no_grad():
        hidden, cell = model.encoder(data)
    
    output_start = PARAM["output_corpus_dict"]['#']
    out_reshape = np.array(output_start).reshape(1,)
    hidden_par = hidden.unsqueeze(0)
    initial_sequence = torch.tensor(out_reshape).to(device)
    beam = [(0.0, initial_sequence, hidden_par)]

    for i in range(PARAM["max_decoder_length"]):
        candidates = []
        for score, seq, hidden in beam:
            if seq[-1].item() == PARAM["output_corpus_dict"]['$']:
                candidates.append((score, seq, hidden))
                continue
            reshape_last = np.array(seq[-1].item()).reshape(1, )
            hdn = hidden.squeeze(0) 
            x = torch.tensor(reshape_last).to(device)
            output, hidden, cell = model.decoder(x, hdn, cell)

            topK_prob, topK_tokens = torch.topk(output, K = PARAM["beam_width"])

            print(topK_prob, topK_tokens)               


            


In [ ]:
# Clip karna he

In [ ]:
PARAM = {
    "beam_width" : 4,
    "max_encoder_length" : max_encoder_length,
    "max_decoder_length" : max_decoder_length,
    "input_corpus_dict" : input_corpus_dict,
    "output_corpus_dict" : output_corpus_dict,
}

In [ ]:
beam_search(PARAM, model, "harsh$" ,device)